# Notebook #2: Plot ERPs

    
This notebook produces basic exploratory plots of the stimulus- and response-locked ERPs.
None of these plots are included in the paper, which reports analyses based on the PCA-filtered data produced later.

In [ ]:
# coding: utf-8
import os
import sys
from glob import glob
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import mne

import sys
sys.path.append('src')
import eegf # My generic EEG Functions
import functions # Specific to this analysis
from functions import topomap, do_rt_comparison, do_threeway_comparison, do_twoway_comparison, do_component

mpl.rcParams['font.size'] = 20
mpl.rcParams['axes.titlesize'] = 'medium'
mpl.rcParams['axes.labelsize'] = 'medium'
mpl.rcParams['xtick.labelsize'] = 'medium'
mpl.rcParams['ytick.labelsize'] = 'medium'
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.edgecolor'] = 'k'
million = 1000000

subjects = [1001, 1002, 1003, 1004,
            1005, 1006,       1008,
            1009, 1010, 1011, 1012,
            1013, 1014, 1015, 1016,
            1017, 1018, 1019, 1020,
            1021]


In [ ]:
%%capture
trial_epochs = mne.read_epochs('data/trial_epo.fif').apply_baseline((-.1, 0.))
response_epochs = mne.read_epochs('data/response_epo.fif').apply_baseline((-2.1, -2))

if os.path.exists('data/trial-csd_epo.fif'):
    trial_epochs_csd = mne.read_epochs('data/trial-csd_epo.fif').apply_baseline((-.1, 0.))
    response_epochs_csd =  mne.read_epochs('data/response-csd_epo.fif').apply_baseline((-2.1, -2))
else:
    trial_epochs_csd = eegf.surface_laplacian(trial_epochs, m=5)
    trial_epochs_csd.save('data/trial-csd_epo.fif')
    response_epochs_csd = eegf.surface_laplacian(response_epochs, m=5)
    response_epochs_csd.save('data/response-csd_epo.fif')

In [ ]:
print([len(e) for e in [trial_epochs, response_epochs, trial_epochs_csd, response_epochs_csd]])

In [ ]:
def find_outlier_trials(epochs, thresh=120. / million):
    X = epochs.get_data()[:, :32]
    aX = np.abs(X).max(2).max(1)
    return aX > thresh

trial_rej = find_outlier_trials(trial_epochs, thresh=120./million)
df = trial_epochs.metadata
df['rej'] = trial_rej
print( df.groupby('participant').mean()['rej'])
print( trial_rej.mean())

In [ ]:
resp_rej = find_outlier_trials(response_epochs, thresh=120./million)
df = response_epochs.metadata
df['rej'] = resp_rej
print(df.groupby('participant').mean()['rej'])
print( resp_rej.mean())

In [ ]:
trial_epochs = trial_epochs[trial_rej==False]
trial_epochs_csd = trial_epochs_csd[trial_rej==False]
response_epochs = response_epochs[resp_rej==False]
response_epochs_csd = response_epochs_csd[resp_rej==False]


## Lots and lots of ERPs

The hard work in this notebook is done by the `do_component()` function, defined in `functions.py`.

In [ ]:
help(do_component)

All we need to do is call this function on every channel of the original data, 
and every channel of the CSD data, and save the resulting plots.
The plots end up in `figures/components/raw/` and `figures/components/csd/`.

In [ ]:
%mkdir figures
%mkdir figures/components
%mkdir figures/components/raw/
%mkdir figures/components/csd/
mpl.rcParams['font.size'] = 14

In [ ]:
do_big_computations = False # The next two cells take ages. Set False to skip them.

In [ ]:
if do_big_computations:
    for i in range(32):
        print(i, end=', ')
        weights = np.zeros(32)
        weights[i] = 1
        ch = trial_epochs.ch_names[i]
        fig = do_component(trial_epochs, response_epochs, ch=i, 
                           weights = weights, info=trial_epochs.info,
                           title = 'Raw Signal - Channel %s' % ch)

        fig.savefig('figures/components/raw/%i-%s.svg' % (i, ch))
        plt.close()

In [ ]:
if do_big_computations:
    for i in range(32):
        print(i, end=', ')
        weights = np.zeros(32)
        weights[i] = 1
        ch = trial_epochs.ch_names[i]
        fig = do_component(trial_epochs_csd, response_epochs_csd, ch=i, 
                           weights = weights, info=trial_epochs.info,
                           title = 'CSD Signal - Channel %s' % ch)
        fig.savefig('figures/components/csd/%i-%s.svg' % (i, ch))
        plt.close()

In [ ]:
def get_chan(lbl):
    return trial_epochs.ch_names.index(lbl)
midline_chans = [ch for ch in trial_epochs.ch_names if ch.find('z') > 0]
midline_chans

## Specific Stimulus-locked plots

In [ ]:
eegf.plot_joint(trial_epochs_csd.average(), times=[0., .12, .2, .5, .65]);

In [ ]:
fig, axes = plt.subplots(figsize=(20, 8), ncols=3, nrows=2)
axes = np.concatenate(axes)
for ax, ch in zip(axes, midline_chans):
    plt.sca(ax)
    ax.set_title(ch)
    do_twoway_comparison(trial_epochs_csd, get_chan(ch), variable='action',
                         labels=['Wait', 'Act'], title=ch, crop=(-.5, 2.), baseline=(-.1, 0))
plt.tight_layout(rect = (0, 0, 1, .95))
plt.suptitle('Wait vs Act - Midline Electrodes - CSD EEG')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(20, 8), ncols=3, nrows=2)
axes = np.concatenate(axes)
for ax, ch in zip(axes, midline_chans):
    ax.set_title(ch)
    do_threeway_comparison(trial_epochs_csd, get_chan(ch), ax=ax, title=ch, crop=(-.5, 2.), baseline=(-.1, 0),
                          by_subject=True)
plt.tight_layout(rect = (0, 0, 1, .95))
plt.suptitle('Easy vs Difficult vs Guess - Midline Electrodes - CSD EEG')
plt.show()
    

In [ ]:
fig, axes = plt.subplots(figsize=(20, 8), ncols=3, nrows=2)
axes = np.concatenate(axes)
for ax, ch in zip(axes, midline_chans):
    ax.set_title(ch)
    do_threeway_comparison(trial_epochs_csd, get_chan(ch), ax=ax, title=ch, crop=(-.5, 2.), baseline=(-.1, 0),
                           agg_func=functions.std_by_subject,
                           by_subject=True)
    eegf.flipy() # Return to negative-down
plt.tight_layout(rect = (0, 0, 1, .95))
plt.suptitle('Standard devition of Easy vs Difficult vs Guess - Midline Electrodes - CSD EEG')
plt.show()
    

## Specific Response-locked plots

In [ ]:
eegf.plot_joint(response_epochs_csd.average(), times=[-2, -.1, 0.]);

In [ ]:
fig, axes = plt.subplots(figsize=(20, 8), ncols=3, nrows=2)
axes = np.concatenate(axes)
for ax, ch in zip(axes, midline_chans):
    ax.set_title(ch)
    do_threeway_comparison(response_epochs_csd, get_chan(ch), ax=ax, title=ch, by_subject=True)
plt.tight_layout(rect = (0, 0, 1, .95))
plt.suptitle('Easy vs Difficult vs Guess - Midline Electrodes - CSD EEG')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(20, 8), ncols=3, nrows=2)
axes = np.concatenate(axes)
for ax, ch in zip(axes, midline_chans):
    ax.set_title(ch)
    do_threeway_comparison(response_epochs_csd, get_chan(ch), ax=ax, title=ch, 
                           agg_func=functions.std_by_subject, by_subject=True,
                           baseline=(-.01, .01))
    eegf.flipy() # Return to negative-down
plt.tight_layout(rect = (0, 0, 1, .95))
plt.suptitle('Standard devition of Easy vs Difficult vs Guess - Midline Electrodes - CSD EEG')
plt.show()